In [ ]:

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=300)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OllamaEmbeddings())
retriever = vectordb.as_retriever() 
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"mental_health_search","search for mental health related solutions")
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper= ArxivAPIWrapper(top_k_results=5,doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
tools = [wiki,retriever_tool,arxiv]
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

from langchain_community.llms import Ollama

llm = Ollama(model="llama2")
#prompt template

from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
#agents 
from langchain.agents import create_openapi_agent
agent = create_openapi_agent(llm,tools,prompt)
#agent executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"What is depression? "})